In [68]:
import os ,json 
import pandas as pd 
import traceback

In [69]:
from langchain.chat_models import ChatOpenAI 
from langchain.llms import OpenAI 
from langchain.prompts import PromptTemplate 
from langchain.chains import LLMChain 
from langchain.chains import SequentialChain 
from langchain.callbacks import get_openai_callback
import PyPDF2

In [70]:
# ! You can import environment key by 
# 1. create virtual environament variable using python-dotenv
# 2. import from terminal 
# 3. create .env file and import it
from dotenv import load_dotenv 
load_dotenv()
key = os.getenv("OPENAI_API_KEYS")
model = "gpt-3.5-turbo"
print(key)

sk-proj-tig8DcAC1QxDCRLQPFhYT3BlbkFJq5A2p0A3h9g8LoPbPU64


In [71]:
llm  = ChatOpenAI(openai_api_key = key ,model =model ,temperature=0.5)

In [72]:
#Input Promt Skeletone will used in input prompt
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [73]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [74]:
print(TEMPLATE)


Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}




In [75]:
#This prompt will used in Input
quiz_generation_prompt = PromptTemplate(
    input_variables=['text' ,'number' ,'subject' ,'tone' ,'response_json'] ,
    template = TEMPLATE
)
quiz_generation_prompt

PromptTemplate(input_variables=['number', 'response_json', 'subject', 'text', 'tone'], template='\nText:{text}\nYou are an expert MCQ maker. Given the above text, it is your job to create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. \nMake sure the questions are not repeated and check all the questions to be conforming the text as well.\nMake sure to format your response like  RESPONSE_JSON below  and use it as a guide. Ensure to make {number} MCQs\n### RESPONSE_JSON\n{response_json}\n\n')

In [76]:
quiz_chain = LLMChain(llm= llm ,prompt= quiz_generation_prompt ,output_key='quiz' ,verbose=True)
# quiz_chain.run(text = 'text' ,number ='number' ,subject = 'subject' ,tone ='tone' ,response_json ='response_json')

In [77]:
#Output Template
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [78]:
#This prompt will used in Output
quiz_evzluation_prompt = PromptTemplate(
    input_variables=['subject' ,'quiz'] ,
    template = TEMPLATE2
)
quiz_evzluation_prompt

PromptTemplate(input_variables=['quiz', 'subject'], template='\nYou are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. \nif the quiz is not at per with the cognitive and analytical abilities of the students,update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities\nQuiz_MCQs:\n{quiz}\n\nCheck from an expert English Writer of the above quiz:\n')

In [79]:
review_chain = LLMChain(llm=llm ,prompt= quiz_evzluation_prompt ,output_key='review' ,verbose=True)

In [80]:
#Connnect Both Chain(input and output)
generate_evaluation_chain = SequentialChain(
    chains=[quiz_chain , review_chain ] ,input_variables=['text' ,'number' ,'subject' ,'tone' ,'response_json'],
    output_variables=["quiz", "review"], verbose=True,) 
generate_evaluation_chain

SequentialChain(verbose=True, chains=[LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['number', 'response_json', 'subject', 'text', 'tone'], template='\nText:{text}\nYou are an expert MCQ maker. Given the above text, it is your job to create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. \nMake sure the questions are not repeated and check all the questions to be conforming the text as well.\nMake sure to format your response like  RESPONSE_JSON below  and use it as a guide. Ensure to make {number} MCQs\n### RESPONSE_JSON\n{response_json}\n\n'), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000020D31721D20>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000020D31711480>, temperature=0.5, openai_api_key='sk-proj-tig8DcAC1QxDCRLQPFhYT3BlbkFJq5A2p0A3h9g8LoPbPU64', openai_proxy=''), output_key='quiz'), LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['quiz', 

In [81]:
with open('C:\\Users\\PRAMOD KHAVARE\\MCQGEN\\data.txt' ,'r') as f:
    TEXT= f.read()
TEXT

'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, 

In [82]:
#Serialization of Data (Python dictionary into JSON Formated String)
json.dumps(RESPONSE_JSON)
NUMBER=5 
SUBJECT="biology"
TONE="simple"

In [86]:
with get_openai_callback() as cb:
    resopnse = generate_evaluation_chain(
        {
            'text' : TEXT,'number' :NUMBER ,'subject' :SUBJECT,'tone' :TONE ,'response_json' :json.dumps(RESPONSE_JSON)
        }
    )





> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each define

In [87]:
print(type(resopnse))

<class 'dict'>


In [84]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}") #In dolller

Total Tokens:1345
Prompt Tokens:1003
Completion Tokens:342
Total Cost:0.0021885


In [89]:
from src.mcqgenerator.utils.utils import get_table_data_from_quiz
response_quiz = resopnse['quiz']
print((response_quiz))
# response_quiz = json.loads(response_quiz)
# print(type(response_quiz))
get_table_data_from_quiz(response_quiz)


{
    "1": {
        "mcq": "What is the scientific study of life called?",
        "options": {
            "a": "Chemistry",
            "b": "Biology",
            "c": "Physics",
            "d": "Geology"
        },
        "correct": "b"
    },
    "2": {
        "mcq": "What is the main theme that explains the unity and diversity of life?",
        "options": {
            "a": "Energy processing",
            "b": "Evolution",
            "c": "Cell structure",
            "d": "Genetic encoding"
        },
        "correct": "b"
    },
    "3": {
        "mcq": "What allows organisms to move, grow, and reproduce?",
        "options": {
            "a": "Photosynthesis",
            "b": "Energy processing",
            "c": "Respiration",
            "d": "Digestion"
        },
        "correct": "b"
    },
    "4": {
        "mcq": "What is the process that biologists use to make observations, generate hypotheses, and form conclusions?",
        "options": {
            "a":

[{'MCQ': 'What is the scientific study of life called?',
  'Choices': 'a : Chemistry | | b : Biology | | c : Physics | | d : Geology',
  'Correct': 'b'},
 {'MCQ': 'What is the main theme that explains the unity and diversity of life?',
  'Choices': 'a : Energy processing | | b : Evolution | | c : Cell structure | | d : Genetic encoding',
  'Correct': 'b'},
 {'MCQ': 'What allows organisms to move, grow, and reproduce?',
  'Choices': 'a : Photosynthesis | | b : Energy processing | | c : Respiration | | d : Digestion',
  'Correct': 'b'},
 {'MCQ': 'What is the process that biologists use to make observations, generate hypotheses, and form conclusions?',
  'Choices': 'a : Scientific method | | b : Trial and error | | c : Guesswork | | d : Intuition',
  'Correct': 'a'},
 {'MCQ': 'When did life on Earth emerge?',
  'Choices': 'a : 1.5 billion years ago | | b : 3.7 billion years ago | | c : 5 billion years ago | | d : 10 billion years ago',
  'Correct': 'b'}]

In [ ]:
type(resopnse['quiz'])

str

In [ ]:
quiz = (resopnse['quiz'])
quiz = json.loads(quiz)

In [ ]:
quiz

{'1': {'mcq': 'What is the scientific study of life called?',
  'options': {'a': 'Chemistry',
   'b': 'Biology',
   'c': 'Physics',
   'd': 'Geology'},
  'correct': 'b'},
 '2': {'mcq': 'What is the main theme that explains the unity and diversity of life?',
  'options': {'a': 'Evolution',
   'b': 'Photosynthesis',
   'c': 'Mitosis',
   'd': 'Respiration'},
  'correct': 'a'},
 '3': {'mcq': 'Which component of a cell processes hereditary information encoded in genes?',
  'options': {'a': 'Ribosomes',
   'b': 'Mitochondria',
   'c': 'Nucleus',
   'd': 'Endoplasmic reticulum'},
  'correct': 'c'},
 '4': {'mcq': 'What allows organisms to move, grow, and reproduce?',
  'options': {'a': 'Energy processing',
   'b': 'Cell division',
   'c': 'Photosynthesis',
   'd': 'Osmosis'},
  'correct': 'a'},
 '5': {'mcq': 'What is the process by which biologists make observations, pose questions, perform experiments, and form conclusions?',
  'options': {'a': 'Guessing',
   'b': 'Intuition',
   'c': 'Scien

In [ ]:
quiz_table_data = []
for keys,items in quiz.items():
    mcq = items['mcq']
    options = (items['options'])
    options = " | ".join([f"{key} : {value}" for key ,value in options.items()])
    correct = (items['correct'])
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})
print(quiz_table_data)

[{'MCQ': 'What is the scientific study of life called?', 'Choices': 'a : Chemistry | b : Biology | c : Physics | d : Geology', 'Correct': 'b'}, {'MCQ': 'What is the main theme that explains the unity and diversity of life?', 'Choices': 'a : Evolution | b : Photosynthesis | c : Mitosis | d : Respiration', 'Correct': 'a'}, {'MCQ': 'Which component of a cell processes hereditary information encoded in genes?', 'Choices': 'a : Ribosomes | b : Mitochondria | c : Nucleus | d : Endoplasmic reticulum', 'Correct': 'c'}, {'MCQ': 'What allows organisms to move, grow, and reproduce?', 'Choices': 'a : Energy processing | b : Cell division | c : Photosynthesis | d : Osmosis', 'Correct': 'a'}, {'MCQ': 'What is the process by which biologists make observations, pose questions, perform experiments, and form conclusions?', 'Choices': 'a : Guessing | b : Intuition | c : Scientific method | d : Magic', 'Correct': 'c'}]


In [ ]:
final_quiz = pd.DataFrame(
    quiz_table_data
)
final_quiz.to_csv('biology_quiz.csv' ,index=False)

In [ ]:

abc ="""{
    "1": {
        "mcq": "What is the scientific study of life called?",
        "options": {
            "a": "Physics",
            "b": "Chemistry",
            "c": "Biology",
            "d": "Geology"
        },
        "correct": "c"
    },
    "2": {
        "mcq": "Which theme in biology explains the unity and diversity of life?",
        "options": {
            "a": "Evolution",
            "b": "Energy processing",
            "c": "Cell structure",
            "d": "Regulation of internal environments"
        },
        "correct": "a"
    },
    "3": {
        "mcq": "What is the level of organization in biology that studies the anatomy and physiology of plants and animals?",
        "options": {
            "a": "Molecular biology",
            "b": "Cell biology",
            "c": "Population biology",
            "d": "Organismal biology"
        },
        "correct": "d"
    }
}"""

In [ ]:
def min_product_modulo(n, array):
    MOD = 10**9 + 7
    
    # Function to compute GCD of two numbers
    def gcd(a, b):
        while b:
            a, b = b, a % b
        return a
    
    # Compute the GCD of the entire array
    overall_gcd = array[0]
    for i in range(1, n):
        overall_gcd = gcd(overall_gcd, array[i])
        if overall_gcd == 1:
            break  # Early exit if GCD becomes 1
    print(overall_gcd)
    # Compute overall_gcd^n % MOD
    result = 1
    for _ in range(n):
        result = (result * overall_gcd) % MOD

    return result

In [ ]:
def gcd(a, b):
        while b:
            a, b = b, a % b
        return a
list1 =[2,4,9 ,18]
from functools import reduce 
reduce(gcd ,list1)
overall_gcd = list1[0]
for i in range(1,len(list1)+1):
      print(i)

1
2
3
4


In [1]:
def read_file(file):
    """
    This function will help to read file ypu get from streamlit only
    """
    if file.name.endswith(".pdf"):
            pdf_reader =  PyPDF2.PdfReader(file)
            texts = ""
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                texts += page.extract_text()
                print(texts)
            return texts
        
    elif file.name.endswith(".txt"):
            print(file.getvalue().decode('utf-8'))
            return file.getvalue().decode('utf-8')
            
       

In [ ]:
def read_file(file):
    """
    This function will help to read file you get from streamlit only
    """
    if file.name.endswith(".pdf"):
        pdf_reader = PyPDF2.PdfReader(file)
        texts = ""
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            texts += page.extract_text()
        print(texts)
        return texts
        
    elif file.name.endswith(".txt"):
        text = file.getvalue().decode('utf-8')
        print(text)
        return text
    
    else:
        return "Unsupported file format"

2024-06-13 15:49:34.226 
  command:

    streamlit run c:\Users\PRAMOD KHAVARE\MCQGEN\env\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
